***GENERATED CODE FOR hospitallosprediction PIPELINE.***

***DON'T EDIT THIS CODE.***

***CONNECTOR FUNCTIONS TO READ DATA.***

In [ ]:
import os
import datetime
import logging
import warnings
warnings.filterwarnings('ignore')
logging.basicConfig(format='%(levelname)s:%(message)s', level=logging.INFO)


class HDFSConnector:

    def fetch(spark, config):
        ################### INPUT HADOOP HOST PORT TO CONNECT WITH ###############################
        hdfs_server = str(os.environ['HDFS_SERVER'])
        hdfs_port = int(os.environ['HDFS_PORT'])
        df = spark.read.options(header='true', inferschema='true').csv(
            f"hdfs://{hdfs_server}:{hdfs_port}{eval(config)['url']}", header='true')
        display(df.limit(2).toPandas())
        return df

    def put(df, spark, config):
        return df.write.format('csv').options(header='true' if eval(config)["is_header"] == "Use Header Line" else 'false',
                                              delimiter=eval(config)["delimiter"]).save(("%s %s") % (datetime.datetime.now().strftime("%Y-%m-%d %H.%M.%S")+"_", eval(config)['url']))


***SELECTED MODEL PREDICTION FUNCTION***

In [ ]:
import json
import sklearn.metrics
import mlflow.sklearn
import pyspark
import mlflow
import sklearn
from pyspark.sql.types import IntegerType
from pyspark.ml.feature import StringIndexer
from pyspark.sql.functions import col, when
from pyspark.sql.types import IntegerType
from pyspark.sql.functions import mean, stddev, min, max, col


class CleanseData:
    # def __init__(self,df):
    #     #print()

    def cleanValueForFE(self, value):
        if value == None:
            return ""
        elif str(value) == 'nan':
            return "nan"
        else:
            return value

    def replaceByMean(self, feature, df, mean_=-1):
        df1 = df
        df1 = df1.dropna()
        meanValue = self.cleanValueForFE(df1.select(
            mean(col(feature.name)).alias('mean')).collect()[0]["mean"])
        df = df.fillna(meanValue, subset=[feature.name])
        df.withColumn(feature.name, when(col(feature.name) == " ",
                      meanValue).otherwise(col(feature.name).cast("Integer")))
        return df

    def replaceByMax(self, feature, df, max_=-1):
        df1 = df
        df1 = df1.dropna()
        maxValue = self.cleanValueForFE(df1.select(
            max(col(feature.name)).alias('max')).collect()[0]["max"])
        df = df.fillna(maxValue, subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", maxValue).otherwise(col(feature.name)))
        return df

    def replaceByMin(self, feature, df, min_=-1):
        df1 = df
        df1 = df1.dropna()
        minValue = self.cleanValueForFE(df1.select(
            min(col(feature.name)).alias('min')).collect()[0]["min"])
        df = df.fillna(minValue, subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", minValue).otherwise(col(feature.name)))
        return df

    def replaceByStandardDeviation(self, feature, df, stddev_=-1):
        df1 = df
        df1 = df1.dropna()
        stddevValue = self.cleanValueForFE(df1.select(
            stddev(col(feature.name)).alias('stddev')).collect()[0]["stddev"])
        df = df.fillna(stddevValue, subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", stddevValue).otherwise(col(feature.name)))
        return df

    def replaceDateRandomly(self, feature, df):
        df1 = df
        df1 = df1.dropna()
        fillValue = self.cleanValueForFE(
            df.where(col(feature.name).isNotNull()).head(1)[0][feature.name])
        df = df.fillna(str(fillValue), subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", fillValue).otherwise(col(feature.name)))
        # print("CleanseData:replaceDateRandomly Schema : ", df.#printSchema())
        return df

    def replaceNullValues(self, fList, df):
        featuresList = df.schema.fields
        for featureObj in fList:
            for feat in featuresList:
                if featureObj["feature"] in feat.name:
                    featureName = feat
                    if "mean" in featureObj["replaceby"]:
                        df = self.replaceByMean(featureName, df)
                    elif "max" in featureObj["replaceby"]:
                        df = self.replaceByMax(featureName, df)
                    elif "min" in featureObj["replaceby"]:
                        df = self.replaceByMin(featureName, df)
                    elif "stddev" in featureObj["replaceby"]:
                        df = self.replaceByStandardDeviation(featureName, df)
                    elif "random" in featureObj["replaceby"]:
                        df = self.replaceDateRandomly(featureName, df)
        return df


def StringIndexerTransform(df, params, transformationData={}):
    dfReturn = df
    feature = params["feature"]

    dfReturn = dfReturn.fillna({feature: ''})
    outcol = feature + "_stringindexer"
    indexer = StringIndexer(
        inputCol=feature, outputCol=outcol, handleInvalid="skip")
    indexed = indexer.fit(dfReturn).transform(dfReturn)
    dfReturn = indexed
    distinct_values_list = dfReturn.select(
        outcol).distinct().rdd.map(lambda r: r[0]).collect()
    len_distinct_values_list = len(distinct_values_list)
    if len_distinct_values_list <= 4:
        changed_type_df = dfReturn.withColumn(
            outcol, dfReturn[outcol].cast(IntegerType()))
        return changed_type_df
    return dfReturn


class TransformationMain:
    # TODO: change df argument in run with following
    def run(transformationDF, config):
        configObj = json.loads(config)
        featureData = configObj["FE"]
        transformationDF = CleanseData().replaceNullValues(featureData, transformationDF)
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'rcount', 'transformation_label': 'String Indexer'}], 'feature': 'rcount', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
                                                  'count': '366', 'mean': '0.97', 'stddev': '1.35', 'min': '0', 'max': '5+', 'missing': '0', 'distinct': '6'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'rcount'}, {'feature_label': 'rcount', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('rcount')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'gender', 'transformation_label': 'String Indexer'}], 'feature': 'gender', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '366', 'mean': '', 'stddev': '', 'min': 'F', 'max': 'M', 'missing': '0', 'distinct': '2'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'gender'}, {'feature_label': 'gender', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('gender')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'discharged', 'transformation_label': 'String Indexer'}], 'feature': 'discharged', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '366', 'mean': '', 'stddev': '', 'min': '1/10/2012', 'max': '9/7/2012', 'missing': '0', 'distinct': '248'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'discharged'}, {'feature_label': 'discharged', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('discharged')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'facid', 'transformation_label': 'String Indexer'}], 'feature': 'facid', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '366', 'mean': '', 'stddev': '', 'min': 'A', 'max': 'E', 'missing': '0', 'distinct': '5'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'facid'}, {'feature_label': 'facid', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('facid')
        display(transformationDF.limit(2).toPandas())
        return transformationDF


class ModelPredictionClass:

    def __init__(self, sparkDf, runID, label, mType, model_path, subtype):
        self.sparkDf = sparkDf
        self.runID = runID
        self.y = label
        self.yTest = []
        self.yPredicted = []
        self.mType = mType
        self.model_path = model_path
        self.subtype = subtype

    def getmetricsRegression(self):
        scoreMetrics = {}
        scoreMetrics["R2"] = round(
            sklearn.metrics.r2_score(self.yTest, self.yPredicted), 1)
        scoreMetrics["Mean_Squared_Error"] = round(
            sklearn.metrics.mean_squared_error(self.yTest, self.yPredicted), 1)

        return scoreMetrics

    def getmetricsClassification(self):
        scoreMetrics = {}
        self.yPredicted = [int(x) for x in self.yPredicted]
        self.yTest = [int(x) for x in self.yTest]
        scoreMetrics["Accuracy"] = round(
            (100 * sklearn.metrics.accuracy_score(y_true=self.yTest, y_pred=self.yPredicted)), 1)
        scoreMetrics["F1"] = round(
            (100 * sklearn.metrics.f1_score(y_true=self.yTest, y_pred=self.yPredicted, average="weighted")), 1)
        scoreMetrics["Precision"] = round((
            100 * sklearn.metrics.precision_score(y_true=self.yTest, y_pred=self.yPredicted, average="weighted")),
            1)
        scoreMetrics["Recall"] = round((
            100 * sklearn.metrics.recall_score(y_true=self.yTest, y_pred=self.yPredicted, average="weighted")), 1)
        return scoreMetrics

    def getPrediction(self):
        scoreMetrics = {}
        probabilities = {}
        self.sparkDf.persist(pyspark.StorageLevel.MEMORY_AND_DISK)
        self.sparkDf.toPandas()
        data = self.sparkDf.toPandas()
        complete_model_path = "runs:/" + self.runID + "/" + self.model_path
        self.yTest = list(data[self.y])
        data = data.drop(self.y, axis=1).values
        # model get
        sklearn_model = mlflow.sklearn.load_model(
            model_uri=complete_model_path)
        self.yPredicted = sklearn_model.predict(data).tolist()
        if "classification" == self.mType:
            self.yPredicted = [int(i) for i in self.yPredicted]
            scoreMetrics['result'] = self.getmetricsClassification()

            if self.subtype == "naivebayes" or self.subtype == "multinomialNB" or self.subtype == "GaussianNB" or self.subtype == "ComplementNB" or self.subtype == "BernoulliNB" or self.subtype == "CategoricalNB":
                classes = sklearn_model.classes_.tolist()
                scoreMetrics['classes'] = classes
                for i in range(len(classes)):
                    yproba = sklearn_model.predict_proba(data)[:, i].tolist()
                    my_list_rounded = list(
                        map(lambda x: round(x, ndigits=4), yproba))
                    col_name = "Probability_" + str(classes[i])
                    scoreMetrics[col_name] = my_list_rounded
                    probabilities[col_name] = my_list_rounded
                scoreMetrics['yTest'] = self.yTest
                scoreMetrics['yPredicted'] = self.yPredicted
                return scoreMetrics
            else:
                scoreMetrics['yTest'] = self.yTest
                scoreMetrics['yPredicted'] = self.yPredicted
                return scoreMetrics
        elif "regression" == self.mType:
            scoreMetrics['result'] = self.getmetricsRegression()
            scoreMetrics['yTest'] = self.yTest
            scoreMetrics['yPredicted'] = self.yPredicted
            return scoreMetrics

        elif "clustering" == self.mType:
            self.yPredicted = sklearn_model.predict(data).tolist()
            data['prediction'] = self.yPredicted
            return data


***CONNECTOR FUNCTIONS TO WRITE DATA.***

In [ ]:
import datetime
import requests
import datetime
import logging
import warnings
warnings.filterwarnings('ignore')
logging.basicConfig(format='%(levelname)s:%(message)s', level=logging.INFO)


class NumtraConnector:

    def put(inStages, inStagesData, stageId, spark, config):
        path = eval(config)['server_url']
        baseType = eval(config)['baseType']
        results_url = eval(config)['results_url']
        server = eval(config)['server']
        originalfile = eval(config)['orignalKey']
        eval(config)['pathOnly']
        filename = eval(config)['filename']
        eval(config)['ser']
        eval(config)['user']
        eval(config)['password']
        eval(config)['authSource']
        eval(config)['user_id']
        eval(config)['parent_id']
        eval(config)['project_id']
        time = str(int(datetime.datetime.now().timestamp()))

        inStagesData[inStages[0]]

        print(path)
        print(baseType)
        print(results_url)
        print(server)
        print(originalfile)
        print(filename)

        args = {
            'url': path,
            'baseType': baseType,
            'originalfile': originalfile,
            'filename': time + filename
        }

        response = requests.post(results_url, args)
        return response


***READING DATAFRAME***

In [ ]:
############## CREATE SPARK SESSION ############################ ENTER YOUR SPARK MASTER IP AND PORT TO CONNECT TO SERVER ################
from pyspark.sql import SparkSession
spark = SparkSession.builder.master('local[1]').getOrCreate()
#%run hospitallospredictionHooks.ipynb
try:
	#sourcePreExecutionHook()

	hlosforecat = HDFSConnector.fetch(spark, "{'url': '/FileStore/platform/testdata/1700167044722_HLOS_Forecast.csv', 'filename': 'HLOS_Forecast.csv', 'delimiter': ',', 'file_type': 'Delimeted', 'dbfs_token': '', 'dbfs_domain': '', 'FilePath': '/Healthcare/Hospital Length Of Stay/HLOS_Forecast.csv', 'viewFileName': 'HLOS_Forecast.csv', 'is_header': 'Use Header Line', 'baseType': 'hdfs', 'server_url': '/numtraPlatform/NumtraPlatformV3/uploads/platform/', 'results_url': 'http://ml.numtra.com:44040/api/read/hdfs'}")

except Exception as ex: 
	logging.error(ex)
#spark.stop()


***PREDICTING GIVEN DATA WITH SELECTED MODEL***

In [ ]:
#%run hospitallospredictionHooks.ipynb
try:
	#prediction-pipelinePreExecutionHook()

	hlosforecat = TransformationMain.run(hlosforecat,json.dumps( {"FE": [{"transformationsData": [{}], "feature": "AdmitDate", "type": "date", "selected": "True", "replaceby": "random", "stats": {"count": "", "mean": "", "stddev": "", "min": "", "max": "", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "AdmitDate"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "lengthofstay", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "366", "mean": "4.14", "stddev": "2.41", "min": "1", "max": "13", "missing": "0"}, "updatedLabel": "lengthofstay"}, {"transformationsData": [{"feature_label": "rcount", "transformation_label": "String Indexer"}], "feature": "rcount", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "366", "mean": "0.97", "stddev": "1.35", "min": "0", "max": "5+", "missing": "0", "distinct": "6"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "rcount"}, {"transformationsData": [{"feature_label": "gender", "transformation_label": "String Indexer"}], "feature": "gender", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "366", "mean": "", "stddev": "", "min": "F", "max": "M", "missing": "0", "distinct": "2"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "gender"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "dialysisrenalendstage", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "366", "mean": "0.04", "stddev": "0.2", "min": "0", "max": "1", "missing": "0"}, "updatedLabel": "dialysisrenalendstage"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "asthma", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "366", "mean": "0.05", "stddev": "0.23", "min": "0", "max": "1", "missing": "0"}, "updatedLabel": "asthma"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "irondef", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "366", "mean": "0.1", "stddev": "0.31", "min": "0", "max": "1", "missing": "0"}, "updatedLabel": "irondef"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "pneum", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "366", "mean": "0.04", "stddev": "0.19", "min": "0", "max": "1", "missing": "0"}, "updatedLabel": "pneum"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "substancedependence", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "366", "mean": "0.06", "stddev": "0.24", "min": "0", "max": "1", "missing": "0"}, "updatedLabel": "substancedependence"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "psychologicaldisordermajor", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "366", "mean": "0.27", "stddev": "0.44", "min": "0", "max": "1", "missing": "0"}, "updatedLabel": "psychologicaldisordermajo..."}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "depress", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "366", "mean": "0.07", "stddev": "0.26", "min": "0", "max": "1", "missing": "0"}, "updatedLabel": "depress"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "psychother", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "366", "mean": "0.04", "stddev": "0.2", "min": "0", "max": "1", "missing": "0"}, "updatedLabel": "psychother"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "fibrosisandother", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "366", "mean": "0.0", "stddev": "0.05", "min": "0", "max": "1", "missing": "0"}, "updatedLabel": "fibrosisandother"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "malnutrition", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "366", "mean": "0.05", "stddev": "0.22", "min": "0", "max": "1", "missing": "0"}, "updatedLabel": "malnutrition"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "hemo", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "366", "mean": "0.1", "stddev": "0.3", "min": "0", "max": "1", "missing": "0"}, "updatedLabel": "hemo"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "hematocrit", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "366", "mean": "11.88", "stddev": "2.0", "min": "6.666666667", "max": "20.1", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "hematocrit"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "neutrophils", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "366", "mean": "10.0", "stddev": "3.96", "min": "1.1", "max": "30.2", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "neutrophils"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "sodium", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "366", "mean": "137.74", "stddev": "2.9", "min": "128.3622828", "max": "145.7692114", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "sodium"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "glucose", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "366", "mean": "141.66", "stddev": "29.37", "min": "55.5641163", "max": "217.8583832", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "glucose"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "bloodureanitro", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "366", "mean": "13.84", "stddev": "7.89", "min": "4.0", "max": "75.0", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "bloodureanitro"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "creatinine", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "366", "mean": "1.11", "stddev": "0.19", "min": "0.641164433", "max": "1.73327214", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "creatinine"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "bmi", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "366", "mean": "29.89", "stddev": "1.88", "min": "24.69545055", "max": "34.95339678", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "bmi"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "pulse", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "366", "mean": "74.19", "stddev": "11.25", "min": "46", "max": "109", "missing": "0"}, "updatedLabel": "pulse"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "respiration", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "366", "mean": "6.51", "stddev": "0.57", "min": "4.2", "max": "8.7", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "respiration"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "secondarydiagnosisnonicd9", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "366", "mean": "1.9", "stddev": "1.84", "min": "0", "max": "10", "missing": "0"}, "updatedLabel": "secondarydiagnosisnonicd9"}, {"transformationsData": [{"feature_label": "discharged", "transformation_label": "String Indexer"}], "feature": "discharged", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "366", "mean": "", "stddev": "", "min": "1/10/2012", "max": "9/7/2012", "missing": "0", "distinct": "248"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "discharged"}, {"transformationsData": [{"feature_label": "facid", "transformation_label": "String Indexer"}], "feature": "facid", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "366", "mean": "", "stddev": "", "min": "A", "max": "E", "missing": "0", "distinct": "5"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "facid"}]}))

MPRe =  ModelPredictionClass(hlosforecat,903ca4b2eff746fb9c5bfd9e2747249a, None, timeseriesforecasting, platformmodelsArimahospitallosarima_20231116-202839timeseriesforecasting, arima):
responseMetrics = MPRe.getPrediction()
#prediction-pipelinePostExecutionHook(responseMetrics)


except Exception as ex: 
	logging.error(ex)
spark.stop()


***WRITING DATAFRAME***

In [ ]:
#%run hospitallospredictionHooks.ipynb
try:
	#sinkPreExecutionHook()

	hospitallengthofstay = NumtraConnector.fetch(spark, "{'samplefile': None, 'samplecount': None, 'originalcount': None, 'orignalKey': None, 'pathOnly': '/Healthcare/Hospital Length Of Stay', 'project_id': '6512ebcdb91dbaa301dd1375', 'parent_id': '6512ebcdb91dbaa301dd1375', 'original_schema': [], 'actual_schema': [], 'server': 'https://ml.numtra.com:443', 'server_url': '/numtraPlatform/NumtraPlatformV3/uploads/platform/', 'delimiter': ',', 'file_type': 'Delimeted', 'filename': 'HLOSPredicted.csv', 'token': '', 'domain': '', 'is_header': 'Use Header Line', 'url': '/FileStore/platform/uploadedSourceFiles/part-00000-38935313-f464-4b87-8726-d5127352076a-c000.csv', 'results_url': 'http://ml.numtra.com:44040/api/read/hdfs'}")

except Exception as ex: 
	logging.error(ex)
